<a href="https://colab.research.google.com/github/rkdwjdgjs/Test/blob/main/%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B8%B0%EB%A7%90%EA%B3%BC%EC%A0%9C_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 유튜브 맛집 영상 진위 판별: LLM fine-tuning

# **0. 환경 구축**

In [37]:
# 코랩 시작 시 한 번만 실행
!pip install --upgrade pip
!pip install google-api-python-client pandas
!pip install google-api-python-client transformers datasets
!pip install torch
!pip install -q transformers accelerate peft datasets bitsandbytes trl yt_dlp google-api-python-client
!pip install -U bitsandbytes

In [38]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, Dataset
import pandas as pd
import yt_dlp
import json

# **1. LLM 학습용 데이터 수집**





## **1.1 데이터 수집**

*   유튜브 맛집 소개 영상 5개
*   유튜브 API 사용
*   댓글 데이터 수집
*   전처리 X



In [39]:
# 유튜브 댓글 수집 함수

from googleapiclient.discovery import build

def get_youtube_comments(api_key, video_id, max_comments=200):
    youtube = build("youtube", "v3", developerKey=api_key)

    comments = []
    next_page_token = None

    while len(comments) < max_comments:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            maxResults=100,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response["items"]:
            text = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            comments.append(text)
            if len(comments) >= max_comments:
                break

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break

    return comments


## **1.2 댓글 감성 분석**



*   모델: KcELECTRA-base
*   맛집 라벨: 감성분석 score 4점 이상일 경우 맛집
*   train.josnl파일 저장



In [44]:
# 감성 분석 모델 로드

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

sentiment = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0)


Device set to use cuda:0


In [45]:
# 맛집 라벨 조건 설정

def label_to_food(label):
    star = int(label[0])   # "5 stars" → 5
    if star >= 4:
        return "1"    # 맛집
    else:
        return "0"    # 비맛집


In [48]:
# BERT 감성분석 -> 라벨링 -> JSONL파일 저장

import re
import json
import torch


def extract_video_id(url):
    match = re.search(r"v=([^&]+)", url)
    return match.group(1) if match else None


def make_train_jsonl(api_key, url_list, output_path="train.jsonl"):
    with open(output_path, "w", encoding="utf-8") as f:

        for url in url_list:
            video_id = extract_video_id(url)
            if not video_id:
                print("URL에서 videoId 추출 실패:", url)
                continue

            comments = get_youtube_comments(api_key, video_id)
            # -------------------------------
            # 길이 제한으로 잘라내는 함수
            # -------------------------------
            def truncate_text(text, tokenizer, max_len=512):
                tokens = tokenizer.tokenize(text)
                if len(tokens) > max_len:
                    tokens = tokens[:max_len]
                return tokenizer.convert_tokens_to_string(tokens)

            # -------------------------------
            # BERT 감성 분석 (512 제한 포함)
            # -------------------------------
            def safe_sentiment(text, tokenizer, model):
                # 텍스트 길이 줄이기
                text = truncate_text(text, tokenizer)

                inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

                inputs = {k: v.to(model.device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model(**inputs)

                logits = outputs.logits
                probs = torch.softmax(logits, dim=-1)

                label_idx = torch.argmax(probs).item()
                score = probs[0][label_idx].item()

                # huggingface pipeline 형식과 동일하게 반환
                return [{"label": str(label_idx), "score": score}]


            for c in comments:
                res = safe_sentiment(c, tokenizer, model)[0]
                label = label_to_food(res["label"])

                item = {
                    "instruction": "다음 유튜브 댓글이 맛집을 긍정적으로 평가했는지 판별하시오.",
                    "input": c,
                    "output": label
                }

                f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print("JSONL 생성 완료:", output_path)


In [49]:
# 개인 API키 입력
API_KEY = ""

# 학습할 맛집 영상 url
urls = [
    "https://www.youtube.com/watch?v=pgpqzM7Gow8",
    "https://www.youtube.com/watch?v=yRwhSGg9F2g&pp=ygUG66eb7KeR",
    "https://www.youtube.com/watch?v=66_hgJFbDuY&pp=ygUG66eb7KeR",
    "https://www.youtube.com/watch?v=9u_TNpoR2Go&pp=ygUG66eb7KeR",
    "https://www.youtube.com/watch?v=we7qK9zg8rM&pp=ygUG66eb7KeR"
]

make_train_jsonl(API_KEY, urls)


Token indices sequence length is longer than the specified maximum sequence length for this model (790 > 512). Running this sequence through the model will result in indexing errors


JSONL 생성 완료: train.jsonl


# **2. LLM Fine-tuning + SFT**

## **2.1 모델 로드**
*   모델: QLoRA



In [50]:

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import BitsAndBytesConfig

MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 2,179,072 || all params: 1,545,893,376 || trainable%: 0.1410


## **2.2 SFT 파인 튜닝 설정**

In [51]:

from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset


dataset = load_dataset(
    "json",
    data_files="train.jsonl",
    split="train"
)

def format_example(example):
    return f"""[INST] {example['instruction']}

{example['input']} [/INST]

{example['output']}"""

dataset = dataset.map(lambda x: {"text": format_example(x)})

training_args = TrainingArguments(
    output_dir="./out",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    warmup_steps=10,
    max_steps=150,       # 학습량 조절
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args,
    processing_class=tokenizer,
)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/764 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/764 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/764 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/764 [00:00<?, ? examples/s]

In [52]:
# 학습 실행

trainer.train()
model.save_pretrained("./finetuned-model")
tokenizer.save_pretrained("./finetuned-model")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.902400
20,3.272400
30,2.851300
40,2.607700
50,2.353900
60,2.293400
70,2.343800
80,2.240700
90,2.427500
100,2.160300


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


('./finetuned-model/tokenizer_config.json',
 './finetuned-model/special_tokens_map.json',
 './finetuned-model/chat_template.jinja',
 './finetuned-model/vocab.json',
 './finetuned-model/merges.txt',
 './finetuned-model/added_tokens.json',
 './finetuned-model/tokenizer.json')

# **3. 결과**

## 3.1 **테스트용 데이터 로드**

In [53]:
from googleapiclient.discovery import build

youtube = build("youtube", "v3", developerKey=API_KEY)

def get_youtube_comments(video_id, max_comments=50):
    comments = []

    response = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=100,
        textFormat="plainText"
    ).execute()

    while response and len(comments) < max_comments:
        for item in response.get("items", []):
            text = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            comments.append(text)

            if len(comments) >= max_comments:
                break

        if "nextPageToken" not in response:
            break

        response = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=response["nextPageToken"],
            textFormat="plainText"
        ).execute()

    return "\n".join(comments)


## **3.2. 맛집 판별**

In [126]:
# 파인튜닝된 모델 로드
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./finetuned-model", fix_mistral_regex=True)
model = AutoModelForCausalLM.from_pretrained(
    "./finetuned-model",
    device_map="auto"
)

import re

def extract_video_id(url):
    pattern = r"(?:v=|youtu\.be/)([A-Za-z0-9_-]{11})"
    match = re.search(pattern, url)
    return match.group(1) if match else None


def predict_youtube(url):
    video_id = extract_video_id(url)
    if not video_id:
        print("❌ 영상 ID를 추출할 수 없음.")
        return

    comments_text = get_youtube_comments(video_id)

    # 🔹 chat_template 사용 (권장)
    messages = [
        {
            "role": "user",
            "content": f"""아래는 유튜브 영상 댓글들이다. 이 댓글을 기반으로 이 영상이 맛집인지 노맛집인지 판단해라.

출력 형식:
첫 단어는 '맛집' 또는 '노맛집'으로 시작하고, 그 뒤에 한 문장으로 짧게 이유를 설명해라.

예시:
맛집 긍정적인 댓글이 많고 음식이 맛있다는 평가가 많음
노맛집 부정적인 댓글이 많고 서비스가 불친절하다는 의견이 많음

댓글:
{comments_text}"""
        }
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)


    # 🔹 답변 부분만 추출
    if "[/INST]" in decoded:
        content = decoded.split("[/INST]")[-1].strip()
    else:
        # chat_template 사용 시 assistant 답변만 추출
        if "assistant" in decoded.lower():
            content = decoded.split("assistant")[-1].strip()
        else:
            content = decoded.strip()

    # [INST] 태그가 남아있으면 제거
    if content.startswith("[INST]"):
        content = content.replace("[INST]", "").strip()

    content = content.replace("\n", " ").strip()

    words = content.split()
    result_word = words[0] if words else "판단 불가"
    reason = " ".join(words[1:]) if len(words) > 1 else "설명 없음"

    print("🍽️ 최종 판단:", result_word)
    print("📝 판단 이유:", reason)


In [129]:
# 테스트 실행

# 맛집 으로 보이는 영상 url 입력
test_url = "https://www.youtube.com/watch?v=vJT_Hs4VHMI&pp=ygUN66eb7KeRIOyGjOqwnA%3D%3D"
predict_youtube(test_url)

🍽️ 최종 판단: 맛집
📝 판단 이유: 긍정적인 댓글이 많고, 많은 사람들에게推荐되어 보기에 좋다는 평가가 많다. 특히, 여러 가지 다양한 음식을 제공하며, 가격과 가성비도 매우 우수하다는 점이 눈에 띈다. 또한, 많은 사람들에게 추천받아 많은 관심을 받고 있는 모습이 보인다. 이러한 요소들을 종합하면 이 영상이 맛집으로 판단된다.


In [130]:
# 맛집과 관련 없는 영상 url 입력
test_url = "https://www.youtube.com/watch?v=9HMo07DbCq0"
predict_youtube(test_url)

🍽️ 최종 판단: 노맛집
📝 판단 이유: 부정적인 댓글이 많고, 구독자를 위한 레포트가 아닌 일반적인 텍스트로만 작성되어 있어서 판단하기 어렵습니다. 그러나 해당 지역을 방문한 만큼 다양한 정보를 수집하는 것이 중요합니다.
